In [1]:
# import libraries

import pandas as pd
import numpy as np
import os
import glob
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder

In [2]:
# 2021 data processing

df2021 = pd.read_excel("財務數據excel/2021data.xls", header=7)
# Extract the stock ticker symbol from the 'Exchange:Ticker' column
df2021['Exchange:Ticker'] = df2021['Exchange:Ticker'].str.split(':').str[-1]
# Drop the following columns
df2021 = df2021.drop(columns=["Company Name", 
                        "SIC Codes", 
                        "SIC Codes (Primary Code Only)",
                        "SIC Codes (Primary)", 
                        "Company Type",
                        "Total Trading Assets, Dom. - Capital IQ [CY 2021] ($USDmm, Historical rate)",
                        "Retained Earnings - Capital IQ [CY 2021] ($USDmm, Historical rate)",
                        # 2022 column does not exist
                        "Cash Dividends - Compustat [CY 2021] ($USDmm, Historical rate)"])

rating_list = ['A', 
                'A+',
                'A-', 
                'AA',
                'AA+',
                'AA-',
                'AAA',
                'B',
                'B+',
                'B-',
                'BB',
                'BB+',
                'BB-',
                'BBB',
                'BBB+',
                'BBB-',
                'CCC',
                'CCC+',
                'CCC-',
                'D']

# Create a rating map dictionary
rating_map = {
    'AAA': 1, 'AA+': 1, 'AA': 1, 'AA-': 1, 'A+': 1, 'A': 1, 'A-': 1,
    'BBB+': 2, 'BBB': 2, 'BBB-': 2,
    'BB+': 3, 'BB': 3, 'BB-': 3,
    'B+': 4, 'B': 4, 'B-': 4, 'CCC+': 4, 'CCC': 4, 'CCC-': 4, 'D': 4,
    'NR': np.nan
}

# Map the credit rating values to numerical values 
df2021['S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating)'] = df2021['S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating)'].map(lambda x: rating_map.get(x, x))

# Replace '-' with NaN values in all columns
for col in df2021.columns:
    df2021[col] = df2021[col].replace('-', np.nan)

# Replace 'NM' with NaN values
df2021["Payout Ratio - Capital IQ [CY 2021] (%)"] = df2021["Payout Ratio - Capital IQ [CY 2021] (%)"].replace('NM', np.nan)

for col in df2021.columns:
    # Fill NaN values with the mode
    if col in ["Exchange:Ticker","S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating)"]:
        df2021[col] = df2021[col].fillna(df2021[col].mode().iloc[0])  
    # Fill NaN values with the mean
    else:
        df2021[col] = df2021[col].fillna(df2021[col].mean())  

# For each ticker in the 'Exchange:Ticker' column, search for a matching text file in the '10-K文字檔/2021data/' directory and add the content to a new 'text' column in the DataFrame
for ticker in df2021['Exchange:Ticker']:
    txt_files = glob.glob(os.path.join('10-K文字檔/2021data/', f"{ticker}_*.txt"))
    if txt_files:
        with open(txt_files[0], 'r') as f:
            content = f.read()
        df2021.loc[df2021['Exchange:Ticker'] == ticker, 'text'] = content
    else:
        df2021.loc[df2021['Exchange:Ticker'] == ticker, 'text'] = np.nan

# Drop any rows where the 'text' column is NaN (i.e., no matching text file was found).     
df2021 = df2021.dropna() 

df2021

,Exchange:Ticker,S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating),"Total Assets - Capital IQ [CY 2021] ($USDmm, Historical rate)","Total Capital - Capital IQ [CY 2021] ($USDmm, Historical rate)","Sales/Sq. Ft., All (Net) - Capital IQ [CY 2021] ($USD, Historical rate)","Total Equity - Capital IQ [LTM] ($USDmm, Historical rate)","Total Enterprise Value [My Setting] [Latest - 3 Year(s)] ($USDmm, Historical rate)","Market Capitalization [My Setting] [Latest - 3 Year(s)] ($USDmm, Historical rate)",Shares Outstanding [My Setting] [Latest - 3 Year(s)] (mm),"EBITDA - Capital IQ [CY 2021] ($USDmm, Historical rate)","Net Income - Capital IQ [CY 2021] ($USDmm, Historical rate)","Operating Income - Capital IQ [CY 2021] ($USDmm, Historical rate)",Cash Dividends to Net Income - Capital IQ [CY 2021] (%),"All Other Identifiable Intangible Assets - Capital IQ [CY 2021] ($USDmm, Historical rate)","Net Working Capital - Capital IQ [CY 2021] ($USD, Historical rate)",Net Working Capital/ Total Assets - Capital IQ [LTM],"Book Value/Share - Capital IQ [CY 2021] ($USD, Historical rate)",Payout Ratio - Capital IQ [CY 2021] (%),text
1,FLWS,2.0,1321.3,877.80000,327.54,485.3,2085.500000,2067.70,65.100,167.100000,90.00,121.0,29.500595,242.347939,-86.800000,0.027000,8.66,53.66197,Item 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...
3,TXG,2.0,1018.8,899.50000,327.54,718.8,16822.300000,17367.00,109.800,-31.800000,-58.20,-52.9,29.500595,242.347939,54.900000,0.106000,7.27,53.66197,Item 7. Management’s Discussion and Analysis o...
5,EFSH,2.0,47.0,33.90000,327.54,-28.3,18.100000,9.47,0.012,-0.195000,-3.31,-1.1,29.500595,242.347939,-1.550000,-0.197000,-85.00,53.66197,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...
7,SRCE,2.0,8096.3,7111.43281,327.54,1081.5,14811.279084,1234.90,25.300,878.501129,118.50,156.8,25.710000,0.060000,19.099566,-0.045218,37.00,26.40000,Item 7. Management’s Discussion and Analysis o...
9,XXII,2.0,76.0,68.40000,327.54,-11.6,663.200000,693.60,0.635,-27.100000,-32.60,-28.3,29.500595,242.347939,-1.870000,-0.254000,97.40,53.66197,Item 7.Management’s Discussion and Analysis of...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4287,ZI,3.0,6852.9,3306.00000,327.54,2005.9,8681.400000,7855.80,185.500,187.400000,116.80,153.4,29.500595,242.347939,-278.400000,-0.055000,4.95,17.00000,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...
4288,ZS,2.0,2422.0,1529.00000,327.54,958.9,22617.100000,23120.70,135.900,-231.100000,-330.70,-268.7,29.500595,242.347939,-448.400000,-0.217000,3.82,53.66197,Item 7. Management’s Discussion and Analysis o...
4289,ZUMZ,2.0,862.0,736.20000,327.54,353.2,1002.500000,1065.00,25.700,180.800000,119.30,157.9,29.500595,242.347939,32.300000,0.108000,22.10,53.66197,Item 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...
4290,ZUO,2.0,441.3,229.40000,327.54,133.7,1713.200000,1830.50,122.200,-56.900000,-99.40,-73.7,29.500595,242.347939,-93.500000,-0.133000,1.33,53.66197,Item 7. Management’s Discussion and Analysis o...


In [3]:
# 2022 data processing

df2022 = pd.read_excel("財務數據excel/2022data.xls", header=7)
# Extract the stock ticker symbol from the 'Exchange:Ticker' column
df2022['Exchange:Ticker'] = df2022['Exchange:Ticker'].str.split(':').str[-1]
# Drop the following columns
df2022 = df2022.drop(columns=["Company Name", 
                        "SIC Codes", 
                        "SIC Codes (Primary Code Only)",
                        "SIC Codes (Primary)", 
                        "Company Type",
                        "Total Trading Assets, Dom. - Capital IQ [FY 2022] ($USDmm, Historical rate)",
                        "Retained Earnings - Capital IQ [FY 2022] ($USDmm, Historical rate)"])

rating_list = ['A', 
                'A+',
                'A-', 
                'AA',
                'AA+',
                'AA-',
                'AAA',
                'B',
                'B+',
                'B-',
                'BB',
                'BB+',
                'BB-',
                'BBB',
                'BBB+',
                'BBB-',
                'CCC',
                'CCC+',
                'CCC-',
                'D']

# Create a rating map dictionary
rating_map = {
    'AAA': 1, 'AA+': 1, 'AA': 1, 'AA-': 1, 'A+': 1, 'A': 1, 'A-': 1,
    'BBB+': 2, 'BBB': 2, 'BBB-': 2,
    'BB+': 3, 'BB': 3, 'BB-': 3,
    'B+': 4, 'B': 4, 'B-': 4, 'CCC+': 4, 'CCC': 4, 'CCC-': 4, 'D': 4,
    'NR': np.nan
}

# Map the credit rating values to numerical values 
df2022['S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating)'] = df2022['S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating)'].map(lambda x: rating_map.get(x, x))

# Replace '-' with NaN values in all columns
for col in df2022.columns:
    df2022[col] = df2022[col].replace('-', np.nan)
    
# Replace 'NM' with NaN values    
df2022["Payout Ratio - Capital IQ [FY 2022] (%)"] = df2022["Payout Ratio - Capital IQ [FY 2022] (%)"].replace('NM', np.nan)

for col in df2022.columns:
    # Fill NaN values with the mode
    if col in ["Exchange:Ticker","S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating)"]:
        df2022[col] = df2022[col].fillna(df2022[col].mode().iloc[0])  
    # Fill NaN values with the mean
    else:
        df2022[col] = df2022[col].fillna(df2022[col].mean())  

# For each ticker in the 'Exchange:Ticker' column, search for a matching text file in the '10-K文字檔/2021data/' directory and add the content to a new 'text' column in the DataFrame
for ticker in df2022['Exchange:Ticker']:
    txt_files = glob.glob(os.path.join('10-K文字檔/2021data/', f"{ticker}_*.txt"))
    if txt_files:
        with open(txt_files[0], 'r') as f:
            content = f.read()
        df2022.loc[df2022['Exchange:Ticker'] == ticker, 'text'] = content
    else:
        df2022.loc[df2022['Exchange:Ticker'] == ticker, 'text'] = np.nan

# Drop any rows where the 'text' column is NaN (i.e., no matching text file was found).    
df2022 = df2022.dropna() 

df2022

,Exchange:Ticker,S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating),"Total Assets - Capital IQ [FY 2022] ($USDmm, Historical rate)","Total Capital - Capital IQ [FY 2022] ($USDmm, Historical rate)","Sales/Sq. Ft., All (Net) - Capital IQ [FY 2022] ($USD, Historical rate)","Total Equity - Capital IQ [LTM] ($USDmm, Historical rate)","Total Enterprise Value [My Setting] [Latest - 3 Year(s)] ($USDmm, Historical rate)","Market Capitalization [My Setting] [Latest - 3 Month(s)] ($USDmm, Historical rate)",Shares Outstanding [My Setting] [Latest - 3 Year(s)] (mm),"EBITDA - Capital IQ [FY 2022] ($USDmm, Historical rate)","Net Income - Capital IQ [FY 2022] ($USDmm, Historical rate)","Operating Income - Capital IQ [FY 2022] ($USDmm, Historical rate)","Cash Dividends - Compustat [FY 2022] ($USDmm, Historical rate)","All Other Identifiable Intangible Assets - Capital IQ [FY 2022] ($USDmm, Historical rate)","Net Working Capital - Capital IQ [FY 2022] ($USD, Historical rate)",Net Working Capital/ Total Assets - Capital IQ [LTM],"Book Value/Share - Capital IQ [FY 2022] ($USD, Historical rate)",Payout Ratio - Capital IQ [FY 2022] (%),text
1,FLWS,2.0,1094.9,808.500000,509.3375,485.3,2085.500000,670.10,65.100,91.000000,29.6,41.90,0.00,255.568462,84.000000,0.027000,7.89,66.514275,Item 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...
3,TXG,2.0,1029.0,900.900000,509.3375,718.8,16822.300000,5554.40,109.800,-138.400000,-166.0,-163.70,0.00,255.568462,83.100000,0.106000,7.00,66.514275,Item 7. Management’s Discussion and Analysis o...
5,EFSH,2.0,45.5,32.500000,509.3375,-28.3,18.100000,8.35,0.012,-3.700000,-10.2,-5.74,-1.85,255.568462,-2.480000,-0.197000,1.19,66.514275,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...
7,SRCE,2.0,8339.4,7120.819053,509.3375,1081.5,14811.279084,1230.60,25.300,904.243858,120.5,160.50,-32.10,0.041000,17.927555,-0.045247,35.00,26.600000,Item 7. Management’s Discussion and Analysis o...
9,XXII,2.0,114.7,97.600000,509.3375,-11.6,663.200000,6.86,0.635,-32.700000,-59.8,-34.00,0.00,255.568462,12.100000,-0.254000,107.00,66.514275,Item 7.Management’s Discussion and Analysis of...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4287,ZI,3.0,7136.4,3585.700000,509.3375,2005.9,8681.400000,6485.30,185.500,222.800000,63.2,183.20,0.00,255.568462,-298.800000,-0.055000,5.62,66.514275,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...
4288,ZS,2.0,2832.7,1619.000000,509.3375,958.9,22617.100000,31731.50,135.900,-290.300000,-390.3,-326.70,0.00,255.568462,-583.000000,-0.217000,4.01,66.514275,Item 7. Management’s Discussion and Analysis o...
4289,ZUMZ,2.0,862.0,736.200000,509.3375,353.2,1002.500000,343.30,25.700,180.800000,119.3,157.90,0.00,255.568462,32.300000,0.108000,22.10,66.514275,Item 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...
4290,ZUO,2.0,441.3,229.400000,509.3375,133.7,1713.200000,1133.30,122.200,-56.900000,-99.4,-73.70,0.00,255.568462,-93.500000,-0.133000,1.33,66.514275,Item 7. Management’s Discussion and Analysis o...


In [4]:
# Rename the columns of df2022 to match df2021
df2022.columns = df2021.columns

In [5]:
# Concatenate the two DataFrames
df = pd.concat([df2021, df2022], ignore_index=True)
df

,Exchange:Ticker,S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating),"Total Assets - Capital IQ [CY 2021] ($USDmm, Historical rate)","Total Capital - Capital IQ [CY 2021] ($USDmm, Historical rate)","Sales/Sq. Ft., All (Net) - Capital IQ [CY 2021] ($USD, Historical rate)","Total Equity - Capital IQ [LTM] ($USDmm, Historical rate)","Total Enterprise Value [My Setting] [Latest - 3 Year(s)] ($USDmm, Historical rate)","Market Capitalization [My Setting] [Latest - 3 Year(s)] ($USDmm, Historical rate)",Shares Outstanding [My Setting] [Latest - 3 Year(s)] (mm),"EBITDA - Capital IQ [CY 2021] ($USDmm, Historical rate)","Net Income - Capital IQ [CY 2021] ($USDmm, Historical rate)","Operating Income - Capital IQ [CY 2021] ($USDmm, Historical rate)",Cash Dividends to Net Income - Capital IQ [CY 2021] (%),"All Other Identifiable Intangible Assets - Capital IQ [CY 2021] ($USDmm, Historical rate)","Net Working Capital - Capital IQ [CY 2021] ($USD, Historical rate)",Net Working Capital/ Total Assets - Capital IQ [LTM],"Book Value/Share - Capital IQ [CY 2021] ($USD, Historical rate)",Payout Ratio - Capital IQ [CY 2021] (%),text
0,FLWS,2.0,1321.3,877.80000,327.5400,485.3,2085.500000,2067.70,65.100,167.100000,90.00,121.0,29.500595,242.347939,-86.800000,0.027000,8.66,53.661970,Item 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...
1,TXG,2.0,1018.8,899.50000,327.5400,718.8,16822.300000,17367.00,109.800,-31.800000,-58.20,-52.9,29.500595,242.347939,54.900000,0.106000,7.27,53.661970,Item 7. Management’s Discussion and Analysis o...
2,EFSH,2.0,47.0,33.90000,327.5400,-28.3,18.100000,9.47,0.012,-0.195000,-3.31,-1.1,29.500595,242.347939,-1.550000,-0.197000,-85.00,53.661970,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...
3,SRCE,2.0,8096.3,7111.43281,327.5400,1081.5,14811.279084,1234.90,25.300,878.501129,118.50,156.8,25.710000,0.060000,19.099566,-0.045218,37.00,26.400000,Item 7. Management’s Discussion and Analysis o...
4,XXII,2.0,76.0,68.40000,327.5400,-11.6,663.200000,693.60,0.635,-27.100000,-32.60,-28.3,29.500595,242.347939,-1.870000,-0.254000,97.40,53.661970,Item 7.Management’s Discussion and Analysis of...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5419,ZI,3.0,7136.4,3585.70000,509.3375,2005.9,8681.400000,6485.30,185.500,222.800000,63.20,183.2,0.000000,255.568462,-298.800000,-0.055000,5.62,66.514275,ITEM 7. MANAGEMENT’S DISCUSSION AND ANALYSIS O...
5420,ZS,2.0,2832.7,1619.00000,509.3375,958.9,22617.100000,31731.50,135.900,-290.300000,-390.30,-326.7,0.000000,255.568462,-583.000000,-0.217000,4.01,66.514275,Item 7. Management’s Discussion and Analysis o...
5421,ZUMZ,2.0,862.0,736.20000,509.3375,353.2,1002.500000,343.30,25.700,180.800000,119.30,157.9,0.000000,255.568462,32.300000,0.108000,22.10,66.514275,Item 7.\nMANAGEMENT’S DISCUSSION AND ANALYSIS ...
5422,ZUO,2.0,441.3,229.40000,509.3375,133.7,1713.200000,1133.30,122.200,-56.900000,-99.40,-73.7,0.000000,255.568462,-93.500000,-0.133000,1.33,66.514275,Item 7. Management’s Discussion and Analysis o...


In [6]:
# Sample 500 rows with a fixed random state
df = df.sample(n=500, random_state=42)

In [7]:
# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the DistilBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModel.from_pretrained("distilbert-base-uncased").to(device)

# Split the DataFrame into training and validation sets
df_train, df_val = train_test_split(df, test_size=0.2, random_state=42)

# Extract the general features from the training and validation sets
general_features_train = df_train.iloc[:,2:-1]
general_features_val = df_val.iloc[:,2:-1]

In [8]:
# Tokenize the text data for the training and validation sets
tokenized_train = tokenizer(df_train["text"].values.tolist(), padding=True, truncation=True, return_tensors="pt")
tokenized_val = tokenizer(df_val["text"].values.tolist(), padding=True, truncation=True, return_tensors="pt")

# Pass the tokenized text through the DistilBERT model to get the hidden states
with torch.no_grad():
    hidden_train = model(**tokenized_train)
    hidden_val = model(**tokenized_val)

# Get only the [CLS] token hidden states
cls_train = hidden_train.last_hidden_state[:,0,:]
cls_val = hidden_val.last_hidden_state[:,0,:]

In [9]:
# Encode the target variable
le = LabelEncoder()

# Concatenate the [CLS] token hidden states and the general features for the training set
x_train = torch.cat((cls_train, torch.from_numpy(general_features_train.values)), 1)
y_train = df_train["S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating)"]
y_train = le.fit_transform(y_train)

# Concatenate the [CLS] token hidden states and the general features for the validation set
x_val = torch.cat((cls_val, torch.from_numpy(general_features_val.values)), 1)
y_val = df_val["S&P Entity Credit Rating - Issuer Credit Rating - Local Currency LT [Latest] (Rating)"]
y_val = le.fit_transform(y_val)

# Print the shapes of the input and target tensors
print(f"x_train shape: {x_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"x_val shape: {x_val.shape}")
print(f"y_val shape: {y_val.shape}")

x_train shape: torch.Size([400, 784])
y_train shape: (400,)
x_val shape: torch.Size([100, 784])
y_val shape: (100,)


In [10]:
# Create the XGBoost model
model = xgb.XGBClassifier(objective='multi:softprob',    # Set the objective function for multi-class classification
                            n_estimators=1000,           # Set the number of boosting iterations
                            max_depth=7,                 # Set the maximum depth of the decision trees
                            learning_rate=0.1,           # Set the learning rate for the boosting algorithm
                            random_state=42)             # Set the random state for reproducibility

# Train the model
model.fit(x_train, y_train)

# Make predictions on the validation set
y_val_pred = model.predict(x_val)

# Calculate evaluation metrics
acc = accuracy_score(y_val, y_val_pred)                             # Calculate the accuracy score
f1 = f1_score(y_val, y_val_pred, average='weighted')                # Calculate the weighted F1-score
precision = precision_score(y_val, y_val_pred, average='weighted')  # Calculate the weighted precision
recall = recall_score(y_val, y_val_pred, average='weighted')        # Calculate the weighted recall
cm = confusion_matrix(y_val, y_val_pred)                            # Calculate the confusion matrix

# Print the evaluation metrics
print(f"Accuracy: {acc:.2f}")
print(f"F1-score: {f1:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print("Confusion Matrix:")
print(cm)

Accuracy: 0.74
F1-score: 0.64
Precision: 0.58
Recall: 0.74
Confusion Matrix:
[[ 2  3  0  0]
 [ 0 72  0  0]
 [ 0 12  0  0]
 [ 0 11  0  0]]


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
